In [1]:
import pandas as pd
cal = pd.read_csv('/data/linq/.qlib/qlib_data/cn_data/calendars/day.txt')
delta = 0
TODAY = str(cal.iloc[-1-delta,0])
YESTERDAY = str(cal.iloc[-2-delta,0])
# TODAY = str(cal.iloc[-2,0])
# YESTERDAY = str(cal.iloc[-3,0])

BT_START = '2024-11-01'

print(BT_START)
print(YESTERDAY)
print(TODAY)

2024-11-01
2024-12-23
2024-12-24


In [2]:
# BT_START = '2024-12-01'
# TODAY = '2024-12-09'

test_split = (TODAY, TODAY)
bt_split = (BT_START, YESTERDAY)
pos_txt = f'''\
date {TODAY}
cash 721.00
SZ000001 2100
SH600570 800
SZ000661 200
SH600115 4800
SZ002459 1300
SZ002129 2000
SZ000002 2300
SH600372 1400
SH600188 1100
'''

In [3]:
import qlib
import pandas as pd
from pandas import Timestamp
import json
from qlib.constant import REG_CN
from qlib.utils import exists_qlib_data, init_instance_by_config
from qlib.workflow import R
from qlib.workflow.record_temp import SignalRecord, PortAnaRecord
from qlib.utils import flatten_dict
from qlib.data.filter import NameDFilter
from qlib.backtest.position import Position
from qlib.data import D

import copy
import pprint
import warnings
warnings.filterwarnings('ignore')

provider_uri = "/data/linq/.qlib/qlib_data/cn_data"  # target_dir
qlib.init(provider_uri=provider_uri, region=REG_CN)

from lilab.qlib.utils.tools import normalize_position_history, load_position_text, load_position_history, save_position_history, fill_price_position_history
from lilab.qlib.backtest.benchmark import BENCH_Step

/home/linq/anaconda3/envs/qlib39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2355796:MainThread](2024-12-24 22:04:19,056) INFO - qlib.Initialization - [config.py:416] - default_conf: client.


[2355796:MainThread](2024-12-24 22:04:19,059) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.


[2355796:MainThread](2024-12-24 22:04:19,060) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/data/linq/.qlib/qlib_data/cn_data')}


Record position history

In [4]:
cur_pos = load_position_text(pos_txt)
norm_cur_pos = normalize_position_history(cur_pos)
norm_cur_pos = fill_price_position_history(norm_cur_pos)
try:
    pos_history = load_position_history()
except:
    pos_history = {}
pos_history.update(norm_cur_pos)
# print(pos_history)
save_position_history(pos_history)
dt, pos = next(iter(cur_pos.items()))
account = pos.position
dt, norm_pos = next(iter(norm_cur_pos.items()))
norm_account = pos.position
hold_lst = pos.get_stock_list()
print(dt.date())
print(account)
print(norm_pos)
print(hold_lst)

save_position_history error
2024-12-24
{'SZ000001': {'amount': 2100}, 'SH600570': {'amount': 800}, 'SZ000661': {'amount': 200}, 'SH600115': {'amount': 4800}, 'SZ002459': {'amount': 1300}, 'SZ002129': {'amount': 2000}, 'SZ000002': {'amount': 2300}, 'SH600372': {'amount': 1400}, 'SH600188': {'amount': 1100}, 'cash': 721.0}
{'_settle_type': 'None', 'position': {'SH600115': {'amount': 28241, 'price': 0.6951503753662109}, 'SZ002459': {'amount': 17480, 'price': 1.0820815563201904}, 'SZ000002': {'amount': 14867, 'price': 1.1958640813827515}, 'SH600188': {'amount': 6609, 'price': 2.3198492527008057}, 'SZ000001': {'amount': 13990, 'price': 1.7802048921585083}, 'SZ002129': {'amount': 4184, 'price': 4.454466342926025}, 'SZ000661': {'amount': 2163, 'price': 9.297245979309082}, 'SH600372': {'amount': 0, 'price': inf}, 'SH600570': {'amount': 3727, 'price': 6.362741470336914}, 'cash': 721.0, 'now_account_value': nan}, 'init_cash': 721.0}
['SH600115', 'SZ002459', 'SZ000002', 'SH600188', 'SZ000001', 'S

In [5]:

pos_history = load_position_history()
hold_day = {}
_norm_pos = norm_pos.__dict__['position']
print(_norm_pos)
dt_lst = sorted(list(pos_history.keys()))[::-1]
for k, v in _norm_pos.items():
    if not isinstance(v, dict):
        continue
    cnt = 0
    for dt in dt_lst:
        if str(dt.date())==TODAY:
            # print(dt.date(), TODAY)
            continue
        tmp_pos = pos_history[dt]
        if k not in tmp_pos.get_stock_list():
            break
        cnt += 1
    # print(k, cnt)
    hold_day[k] = cnt
print(hold_day)

{'SH600115': {'amount': 28241, 'price': 0.6951503753662109}, 'SZ002459': {'amount': 17480, 'price': 1.0820815563201904}, 'SZ000002': {'amount': 14867, 'price': 1.1958640813827515}, 'SH600188': {'amount': 6609, 'price': 2.3198492527008057}, 'SZ000001': {'amount': 13990, 'price': 1.7802048921585083}, 'SZ002129': {'amount': 4184, 'price': 4.454466342926025}, 'SZ000661': {'amount': 2163, 'price': 9.297245979309082}, 'SH600372': {'amount': 0, 'price': inf}, 'SH600570': {'amount': 3727, 'price': 6.362741470336914}, 'cash': 721.0, 'now_account_value': nan}
{'SH600115': 0, 'SZ002459': 2, 'SZ000002': 2, 'SH600188': 0, 'SZ000001': 0, 'SZ002129': 2, 'SZ000661': 0, 'SH600372': 0, 'SH600570': 2}


In [6]:
def simp_eval(res_df, k, t=None, day=None):
    # print(res_df)
    if day:
        date = str(cal.iloc[-day-1,0])
        res_df = res_df[res_df.index.get_level_values('datetime') >= date]
    # Step 1: Group by multiple indices
    grouped = res_df.groupby(['datetime'])

    # Step 2: Sort by 'value' within each group in descending order
    sorted_df = grouped.apply(lambda x: x.sort_values(by='score', ascending=False))
    sorted_df['win'] = sorted_df['label'] * sorted_df['score'] > 0
    # Step 3: Select the top 5 rows for each group
    # top_k_per_group = sorted_df.groupby(['datetime']).head(k)
    top_k_per_group = sorted_df.groupby(['datetime']).apply(lambda x: x.nlargest(k, 'score'))#.reset_index(level=1, drop=1)
    if t:
        # top_k_per_group = top_k_per_group.tail(t)
        top_k_per_group = top_k_per_group.groupby(['datetime']).apply(lambda x: x.nsmallest(t, 'score'))
    # print(top_k_per_group)
    top_k_ic = top_k_per_group['label'].corr(top_k_per_group['score'])
    s, n = top_k_per_group['win'].sum(), len(top_k_per_group)
    # print(s, n)
    top_k_win = 1.0 * top_k_per_group['win'].sum() / len(top_k_per_group)
    return top_k_ic, top_k_win 

def run_model(model, path='/home/linq/finance/qniverse/log'):
    res_df = pd.read_csv(f'{path}/{model}_backtest_return_{TODAY}.csv', index_col=['datetime','instrument'])

    def eval_day(day=None):
        top5ic, top5win = simp_eval(res_df, 5, day=day)
        top10ic, top10win = simp_eval(res_df, 10, 5, day=day)
        # top15ic, top15win = simp_eval(res_df, 15, 5, day=day)
        top20ic, top20win = simp_eval(res_df, 20, 10, day=day)
        # top25ic, top25win = simp_eval(res_df, 25, 5, day=day)
        top30ic, top30win = simp_eval(res_df, 30, 10, day=day)
        top40ic, top40win = simp_eval(res_df, 40, 10, day=day)
        top50ic, top50win = simp_eval(res_df, 50, 10, day=day)
        top60ic, top60win = simp_eval(res_df, 60, 10, day=day)
        top70ic, top70win = simp_eval(res_df, 70, 10, day=day)
        top80ic, top80win = simp_eval(res_df, 80, 10, day=day)        
        top90ic, top90win = simp_eval(res_df, 90, 10, day=day)
        top100ic, top100win = simp_eval(res_df, 100, 10, day=day)
        top110ic, top110win = simp_eval(res_df, 110, 10, day=day)
        top120ic, top120win = simp_eval(res_df, 120, 10, day=day)
        top130ic, top130win = simp_eval(res_df, 130, 10, day=day)
        top140ic, top140win = simp_eval(res_df, 140, 10, day=day)
        top150ic, top150win = simp_eval(res_df, 150, 10, day=day)
        topic, topwin = simp_eval(res_df, 50, day=day)
        # return [topic, topwin, top5ic, top5win, top10ic, top10win, top15ic, top15win, top20ic, top20win, top25ic, top25win, top30ic, top30win]
        return [topwin, top5win, top10win, top20win, top30win, top40win, top50win, top60win, top70win, top80win, top90win, top100win, top110win, top120win, top130win, top140win, top150win], [topic, top5ic, top10ic, top20ic, top30ic, top40ic, top50ic, top60ic, top70ic, top80ic, top90ic, top100ic, top110ic, top120ic, top130ic, top140ic, top150ic]
    day1metrics = eval_day(2)
    day5metrics = eval_day(6)
    day10metrics = eval_day(11)
    day20metrics = eval_day(21)
    
    pred_df = pd.read_csv(f'{path}/{model}_tomorrow_return_{TODAY}.csv', index_col=['datetime','instrument'])

    return day1metrics, day5metrics, day10metrics, day20metrics, pred_df, res_df

# Run

In [7]:
info_win_dict1, info_ic_dict1 = {}, {}
info_win_dict5, info_ic_dict5 = {}, {}
info_win_dict10, info_ic_dict10 = {}, {}
info_win_dict20, info_ic_dict20 = {}, {}
pred_lst = []

name_lst =  ['PatchTST', 'PDF', 'SegRNN', 'TimeBridge', 'XGB', 'TimesNet', 'WFTNet', 'GBDTS']
for model in name_lst:
    NAME = model
    print(model)
    day1metrics, day5metrics, day10metrics, day20metrics, pred_df, res_df = run_model(model)
    info_win_dict1[NAME], info_ic_dict1[NAME] = day1metrics
    info_win_dict5[NAME], info_ic_dict5[NAME] = day5metrics
    info_win_dict10[NAME], info_ic_dict10[NAME] = day10metrics
    info_win_dict20[NAME], info_ic_dict20[NAME] = day20metrics
    # for k in [1, 5, 10, 20]:
    #     eval(f'    info_win_dict{k}[NAME], info_ic_dict{k}[NAME] = day{k}metrics')
    _df = pred_df.loc[:,'score']
    _df.name = model
    # print(_df)
    pred_lst.append(_df)

PatchTST


PDF


SegRNN


TimeBridge


XGB


TimesNet


WFTNet


GBDTS


In [8]:
pred_df = pd.concat(pred_lst, axis=1)

# pred_df = pred_df.loc[:, ~pred_df.columns.duplicated()]
_pred_df = pred_df
pred_df

PatchTST       PDF    SegRNN  TimeBridge       XGB  \
datetime   instrument                                                       
2024-12-24 SH600372    0.003510 -0.002432 -0.003994   -0.006618 -0.062945   
           SZ002916    0.002633  0.002190 -0.003000   -0.000558 -0.311533   
           SH688256    0.002632 -0.000610 -0.001824   -0.003933 -0.274102   
           SZ300442    0.002555  0.001112 -0.002748   -0.003820 -0.179083   
           SH601877    0.002265 -0.001009 -0.000214   -0.006048 -0.001520   
...                         ...       ...       ...         ...       ...   
           SZ002460    0.000280  0.006391 -0.002520    0.000929 -0.030067   
           SZ002311    0.000199  0.004092 -0.001116   -0.002559 -0.013462   
           SH601766    0.000137  0.002907 -0.000773   -0.007194  0.028380   
           SH688599    0.000128  0.003779 -0.001687   -0.005653  0.005691   
           SH601698    0.000068 -0.000621 -0.001250   -0.002624 -0.027251   

                       TimesNet    WFTNet         GBDTS  
datetime   instrument                                    
2024-12-24 SH600372   -0.010944 -0.000688  1.048856e-05  
           SZ002916   -0.010713 -0.003951 -8.862777e-02  
           SH688256   -0.000735 -0.006132 -9.011788e-02  
           SZ300442   -0.008737 -0.006149 -5.821035e-02  
           SH601877   -0.034777 -0.014387  2.618650e-08  
...                         ...       ...           ...  
           SZ002460    0.027414  0.006951 -3.102869e-02  
           SZ002311   -0.006120 -0.011766 -2.417634e-02  
           SH601766    0.037081 -0.000717 -2.625666e-03  
           SH688599   -0.013664 -0.010373  3.517192e-07  
           SH601698    0.010131 -0.000408 -2.601635e-02  

[312 rows x 8 columns]

In [9]:

pred_df['mean'] = _pred_df.mean(axis=1)
pred_df['std'] = _pred_df.std(axis=1)
pred_df['score'] = pd.concat([_pred_df[n]*info_win_dict5[n][0]/_pred_df[n].std() for n in name_lst], axis=1).sum(axis=1)
# pred_df

In [10]:
rank_df = pred_df.droplevel(0).sort_values(by='score', ascending=False)
fields = ['$close/$factor']
_df = D.features(rank_df.index.values, fields, start_time=TODAY, end_time=TODAY, freq='day')
_df.columns = ['close']
_df = _df.droplevel(1)
rank_df = pd.concat([rank_df,_df],axis=1,join='inner')
report_top = rank_df

In [11]:
hold_lst

['SH600115',
 'SZ002459',
 'SZ000002',
 'SH600188',
 'SZ000001',
 'SZ002129',
 'SZ000661',
 'SH600372',
 'SH600570']

In [12]:
pos_rank = rank_df.sort_values(by='score', ascending=True)
pos_rank = pos_rank[pos_rank.index.isin(hold_lst)]
hd = [hold_day[s] for s in pos_rank.index.values]
pos_rank['HD'] = hd
report_pos = pos_rank

In [13]:
color_lst = [n for n in report_pos.columns if n not in ['close', 'HD']]
# color_lst

In [14]:
sty_report_pos = report_pos.style.background_gradient(cmap='Blues_r', subset=color_lst)
sty_report_pos

,PatchTST,PDF,SegRNN,TimeBridge,XGB,TimesNet,WFTNet,GBDTS,mean,std,score,close,HD
instrument,,,,,,,,,,,,,
SH600115,0.000868,0.001621,-0.000524,-0.007048,-0.014072,-0.030473,-0.013094,-0.011649,-0.009296,0.009973,-2.078014,4.090000,0
SZ000002,0.000842,-0.005039,-0.000993,-0.006065,0.011036,-0.002638,-0.001536,0.000000,-0.000549,0.004906,-1.486541,7.730000,2
SZ000661,0.000926,-0.003975,-0.000619,-0.006278,-0.042701,0.019306,-0.006700,-0.000639,-0.005085,0.016167,-1.433097,100.589996,0
SZ002129,0.001756,0.001661,-0.001951,-0.007055,-0.009610,-0.002902,-0.013525,0.000000,-0.003953,0.005221,-1.129743,9.320001,2
SH600372,0.003510,-0.002432,-0.003994,-0.006618,-0.062945,-0.010944,-0.000688,0.000010,-0.010513,0.020237,-0.896267,nan,0
SZ000001,0.000603,0.002972,-0.000430,-0.003726,-0.023619,-0.019499,-0.003861,-0.007305,-0.006858,0.009037,-0.664379,11.860001,0
SH600570,0.001919,-0.003183,-0.001471,-0.006225,-0.056514,0.003528,0.003776,-0.002655,-0.007603,0.018775,-0.545428,29.650000,2
SH600188,0.001952,0.004757,-0.001927,-0.005434,-0.068753,-0.006600,0.004264,-0.036053,-0.013474,0.024228,-0.235809,13.940000,0
SZ002459,0.002037,0.003188,-0.001565,-0.001861,0.002665,0.000656,-0.001432,0.000000,0.000461,0.001875,1.268563,14.550001,2


In [15]:
sty_report_top0 = report_top.head(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top1 = report_top.head(20).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top2 = report_top.head(30).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top3 = report_top.head(40).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top4 = report_top.head(50).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top5 = report_top.head(60).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top6 = report_top.head(70).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top7 = report_top.head(80).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top8 = report_top.head(90).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top9 = report_top.head(100).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top10 = report_top.head(110).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top11 = report_top.head(120).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top12 = report_top.head(130).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top13 = report_top.head(140).tail(10).style.background_gradient(axis=0, subset=color_lst)
sty_report_top14 = report_top.head(150).tail(10).style.background_gradient(axis=0, subset=color_lst)

In [16]:
sty_report_top0

,PatchTST,PDF,SegRNN,TimeBridge,XGB,TimesNet,WFTNet,GBDTS,mean,std,score,close
instrument,,,,,,,,,,,,
SH600803,0.001420,0.009367,-0.000495,0.006559,0.009161,-0.017127,-0.004384,-0.013384,-0.001110,0.009346,2.549125,20.039999
SH603392,0.001653,0.006596,-0.001455,-0.000320,0.005836,0.006385,0.006368,0.000000,0.003133,0.003266,2.519715,74.790001
SH601865,0.001210,0.004967,-0.002016,0.002683,0.045377,-0.017142,0.011261,0.000001,0.005793,0.016766,2.147039,21.200001
SH601985,0.001058,0.003430,-0.000491,-0.000060,0.077705,-0.003129,0.007423,-0.016644,0.008662,0.026909,1.982178,10.360001
SZ000596,0.001685,0.000626,-0.001363,0.004372,-0.039526,0.035115,0.003368,-0.012525,-0.001031,0.019298,1.850064,176.149994
SH601689,0.001362,0.001946,-0.000973,-0.001045,-0.024676,0.031899,0.005742,-0.001549,0.001588,0.014358,1.789015,50.199997
SZ003816,0.001185,0.002342,-0.000664,-0.000916,0.078371,-0.007435,0.003789,0.000000,0.009584,0.026185,1.731931,4.160000
SZ002555,0.000646,0.003388,-0.000540,-0.000653,-0.023842,0.034135,0.004632,0.000000,0.002221,0.014698,1.696963,16.279999
SH601615,0.001644,0.000709,-0.000551,0.004537,-0.000017,-0.028120,0.005541,0.000000,-0.002032,0.010078,1.646417,13.060000


In [17]:
sty_report_top1

,PatchTST,PDF,SegRNN,TimeBridge,XGB,TimesNet,WFTNet,GBDTS,mean,std,score,close
instrument,,,,,,,,,,,,
SZ300999,0.001327,0.003620,-0.000898,-0.002884,-0.001351,0.004117,0.005727,-0.001701,0.000995,0.002970,1.458100,34.200001
SH600875,0.001561,0.005476,-0.000165,-0.001672,-0.006723,-0.002775,-0.007247,0.000000,-0.001443,0.003937,1.365933,15.360000
SH601360,0.000808,0.001198,-0.001781,0.000758,0.144282,0.011948,-0.002654,0.000000,0.019320,0.047417,1.347302,11.140000
SZ000538,0.001260,0.001916,-0.000817,0.001920,0.011470,-0.003625,0.001918,-0.004786,0.001157,0.004612,1.316678,60.250004
SZ002459,0.002037,0.003188,-0.001565,-0.001861,0.002665,0.000656,-0.001432,0.000000,0.000461,0.001875,1.268563,14.550001
SH601618,0.000518,0.004578,-0.000766,0.002175,-0.001333,0.016797,-0.000347,-0.010457,0.001396,0.007109,1.256363,3.370000
SH601878,0.000761,0.005328,0.000111,-0.001691,-0.005888,0.000734,-0.002324,0.000000,-0.000371,0.002986,1.233323,12.660000
SH600887,0.001281,0.001566,-0.000254,-0.002286,-0.026529,-0.000647,0.006803,-0.000816,-0.002610,0.009394,1.224810,30.099998
SH600941,0.001770,0.004469,-0.000801,-0.007155,-0.067192,0.002639,0.008564,0.000000,-0.007213,0.023059,1.202749,114.889992


In [18]:
sty_report_top2

,PatchTST,PDF,SegRNN,TimeBridge,XGB,TimesNet,WFTNet,GBDTS,mean,std,score,close
instrument,,,,,,,,,,,,
SZ002812,0.000949,-0.000847,-0.001523,0.006320,-0.000049,0.028314,-0.001923,0.000000,0.003905,0.009535,1.191217,33.579998
SH600760,0.000495,0.006961,-0.000311,-0.000826,0.006989,0.025058,-0.010227,-0.004217,0.002990,0.009856,1.145837,48.580002
SH600104,0.001128,0.005448,-0.000043,0.002183,-0.041723,0.000537,-0.000529,-0.030072,-0.007884,0.016523,1.144836,18.620001
SH600332,0.001347,0.002505,-0.000820,0.000929,0.010120,-0.004250,0.001873,-0.015116,-0.000426,0.006714,1.109909,28.900002
SZ300316,0.001713,0.002334,0.000273,0.000624,-0.092650,0.038649,-0.003228,-0.037948,-0.011279,0.036262,1.057849,34.180000
SH601872,0.001465,0.001518,-0.000338,-0.002624,-0.009979,0.033692,-0.008365,0.000000,0.001921,0.012682,1.034990,6.530000
SZ300763,0.002211,0.001382,-0.000416,-0.001922,-0.037449,-0.001394,-0.004284,-0.000469,-0.005292,0.012296,1.034563,65.930000
SZ000425,0.000917,0.001200,-0.000057,0.004896,-0.017498,-0.010244,-0.002964,0.000000,-0.002969,0.006853,1.002375,7.710000
SZ002027,0.000751,0.004810,-0.000035,-0.001121,-0.017558,0.035398,-0.009235,-0.012242,0.000096,0.015089,0.983808,7.130000


In [19]:
import dataframe_image as dfi

infodf = pd.DataFrame({'label': ['Model update date', 'Prediction generation date'],
                       'date': ['2024-12-11', TODAY]})

# dfi.export(infodf, './tmp/ensemble_all_info.png',table_conversion='matplotlib')

# dfi.export(sty_report_info5, './tmp/ensemble_all_performance5.png',table_conversion='matplotlib')
# dfi.export(sty_report_info10, './tmp/ensemble_all_performance10.png',table_conversion='matplotlib')
# dfi.export(sty_report_info20, './tmp/ensemble_all_performance20.png',table_conversion='matplotlib')
dfi.export(sty_report_top0, './tmp/ensemble_all_top0.png',table_conversion='matplotlib')
dfi.export(sty_report_top1, './tmp/ensemble_all_top1.png',table_conversion='matplotlib')
dfi.export(sty_report_top2, './tmp/ensemble_all_top2.png',table_conversion='matplotlib')
dfi.export(sty_report_top3, './tmp/ensemble_all_top3.png',table_conversion='matplotlib')
dfi.export(sty_report_top4, './tmp/ensemble_all_top4.png',table_conversion='matplotlib')
dfi.export(sty_report_top5, './tmp/ensemble_all_top5.png',table_conversion='matplotlib')
# dfi.export(sty_report_top6, './tmp/ensemble_all_top6.png',table_conversion='matplotlib')
# dfi.export(sty_report_top7, './tmp/ensemble_all_top7.png',table_conversion='matplotlib')
# dfi.export(sty_report_top8, './tmp/ensemble_all_top8.png',table_conversion='matplotlib')
# dfi.export(sty_report_top9, './tmp/ensemble_all_top9.png',table_conversion='matplotlib')
# dfi.export(sty_report_top10, './tmp/ensemble_all_top10.png',table_conversion='matplotlib')
# dfi.export(sty_report_top11, './tmp/ensemble_all_top11.png',table_conversion='matplotlib')
# dfi.export(sty_report_top12, './tmp/ensemble_all_top12.png',table_conversion='matplotlib')
# dfi.export(sty_report_top13, './tmp/ensemble_all_top13.png',table_conversion='matplotlib')
# dfi.export(sty_report_top14, './tmp/ensemble_all_top14.png',table_conversion='matplotlib')
dfi.export(sty_report_pos, './tmp/ensemble_all_watch.png',table_conversion='matplotlib')

In [20]:

for k in [1,5,10,20]:
    sty_report_win_info = eval(f"pd.DataFrame(info_win_dict{k}, index=['TOP50WIN', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110', '110-120', '120-130', '130-140', '140-150'])").style.background_gradient(axis=None)
    sty_report_ic_info = eval(f"pd.DataFrame(info_ic_dict{k}, index=['TOP50IC', '0-5', '5-10', '10-20', '20-30', '30-40', '40-50', '50-60', '60-70', '70-80', '80-90', '90-100', '100-110', '110-120', '120-130', '130-140', '140-150'])").style.background_gradient(axis=None)
    dfi.export(sty_report_win_info, f'./tmp/ensemble_all_win{k}.png',table_conversion='matplotlib')
    dfi.export(sty_report_ic_info, f'./tmp/ensemble_all_ic{k}.png',table_conversion='matplotlib')

In [21]:
dt_lst = sorted(list(pos_history.keys()))
cash, sv, tot = [], [], []
for dt in dt_lst:
    pos: Position = pos_history[dt]
    cash.append(pos.get_cash())
    sv.append(pos.calculate_stock_value())
    tot.append(pos.calculate_value())
res = pd.DataFrame({'Cash':cash, 'Stock Value':sv, 'Total Value':tot}, index=[d.date() for d in dt_lst])
res['Balance'] = res['Total Value'] - res['Total Value'].shift(1) 
res['Return'] = res['Balance'] / res['Total Value']
res

,Cash,Stock Value,Total Value,Balance,Return
2024-12-12,316.0,177556.186527,177872.186527,NaN,NaN
2024-12-13,316.0,139894.741160,140210.741160,-37661.445368,-0.268606


In [22]:
dfi.export(res, './tmp/private_account.png',table_conversion='matplotlib')

In [23]:
infodf = pd.DataFrame({'label': ['Update date', 'Backtest start date'],
                    'value': [TODAY, BT_START]})

dfi.export(infodf, f'./tmp/summary_info.png',table_conversion='matplotlib')